In [145]:
# Import the vairables we'll be working with. 
import altair as alt
from vega_datasets import data
import pandas as pd
# Read in my annual emmissions data set
annual_emissions_per_capita_chart = pd.read_csv("/home/cccp/Documents/DTSA_5504_Datasets/co-emissions-per-capita-2024.csv")
# Rename the columns I'm working for. 
annual_emissions_per_capita_chart_renamed = annual_emissions_per_capita_chart.rename(columns={"Entity": "Country","Annual CO₂ emissions (per capita)":"CO2_Emissions_Per_Capita"})
# Read in the ISO Codes for the map. 
ISO_Codes = pd.read_csv("/home/cccp/Documents/DTSA_5504_Datasets/ISO_Code.csv")
# Rename the vairables to something more familiar. 
ISO_Codes_Renamed = ISO_Codes.rename(columns={"name": "Country","country-code":"uid","alpha-3":"Code"})
# Merge two datasets together. 
Annual_Emissions_Chart_ISO_Codes_Merged = pd.merge(annual_emissions_per_capita_chart_renamed, ISO_Codes_Renamed, on="Code")
# Drop the unneeded vairables.
Annual_Emissions_Chart_ISO_Codes_Drop_Vairables = Annual_Emissions_Chart_ISO_Codes_Merged.drop(['Year', 'time', 'alpha-2', 'intermediate-region','intermediate-region-code','sub-region-code','region-code','iso_3166-2'], axis = 1)
# Rename the vairables in the new column 
Annual_Emissions_Chart_ISO_Codes_Drop_Vairables_Renamed = Annual_Emissions_Chart_ISO_Codes_Drop_Vairables.rename(columns={"Country_x": "Country","country-code":"uid","alpha-3":"Code"})
# Drop more uneeded variables. 
Annual_Emissions_Chart_ISO_Codes_Drop_Vairables_V2 = Annual_Emissions_Chart_ISO_Codes_Drop_Vairables_Renamed.drop(['Country_y','Code','region','sub-region'], axis = 1)
# Confirm my vairables left
Annual_Emissions_Chart_ISO_Codes_Drop_Vairables_V2.head()

,Country,CO2_Emissions_Per_Capita,uid
0,Afghanistan,0.253848,4
1,Albania,1.591990,8
2,Algeria,4.233817,12
3,Andorra,5.181661,20
4,Angola,0.589497,24


In [138]:
# Data generators for the background
sphere = alt.sphere()
graticule = alt.graticule()

# Create the dropdown binding
dropdown = alt.binding_select(options=country_options, name="Select a country:")

# Create the selection
# selection = alt.selection_single(fields=["Country"], bind=dropdown)

# Source of land data
source = alt.topo_feature(data.world_110m.url, 'countries')

# Layering everything together.
alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='lightblue'),
    alt.Chart(graticule).mark_geoshape(stroke='white', strokeWidth=0.5),
    alt.Chart(source).mark_geoshape(stroke='black').transform_lookup(
        lookup='id',
        from_=alt.LookupData(data=Annual_Emissions_Chart_ISO_Codes_Drop_Vairables_V2, 
                           key='uid', 
                           fields=['CO2_Emissions_Per_Capita', 'Country'])  # Added 'Country'
    ).encode(
        alt.Color('CO2_Emissions_Per_Capita:Q'),
        # Add tooltip for mouseover
        alt.Tooltip(['Country:N', 'CO2_Emissions_Per_Capita:Q']))
).project(
    'naturalEarth1'
).properties(title = "2024 CO2 Emissions Per Capita, Tons Per Person Per Year", width=1920, height=1080).configure_view(stroke=None)

alt.LayerChart(...)